# Работа с демонстрационной БД «Авиаперевозки» в PostGre

Вам предоставляется демонстрационная БД, развернутая в СУБД PostGre.  
Режим доступа - только на чтение.


In [ ]:
import os
import json
import psycopg2
from psycopg2.extras import DictCursor

In [ ]:
# ✅ Устанавливаем переменные окружения прямо в ноутбуке
os.environ['POSTGRESQL_USER'] = 'reader'
os.environ['POSTGRESQL_PASSWORD'] = 'Miba2021'

In [ ]:
# ✅ Используем localhost — БД запущена прямо в Codespaces
POSTGRESQL_HOST = 'localhost'

In [ ]:
# Проверка переменных окружения
print(f"POSTGRESQL_USER={os.environ['POSTGRESQL_USER']}")
print(f"POSTGRESQL_PASSWORD={os.environ['POSTGRESQL_PASSWORD']}")
print(f"POSTGRESQL_HOST={POSTGRESQL_HOST}")

## Создание подключения к БД

In [ ]:
conn = psycopg2.connect(
    dbname='demo',
    user=os.environ['POSTGRESQL_USER'],
    password=os.environ['POSTGRESQL_PASSWORD'],
    host=POSTGRESQL_HOST
)
cur = conn.cursor()
print('✅ Подключение успешно!')

## Получение параметров подключения

In [ ]:
conn.get_dsn_parameters()

## Первые 5 записей из таблицы seats

In [ ]:
query = 'SELECT * FROM bookings.seats LIMIT 5'
cur.execute(query)
records = cur.fetchall()
cur.close()
conn.close()
records

## Конструкция with as — подключение и запрос

In [ ]:
with psycopg2.connect(
    dbname='demo',
    user=os.environ['POSTGRESQL_USER'],
    password=os.environ['POSTGRESQL_PASSWORD'],
    host=POSTGRESQL_HOST
) as conn:
    with conn.cursor() as cur:
        cur.execute('SELECT * FROM bookings.seats LIMIT 10')
        records = cur.fetchall()
records

## Словарь с запросами для получения информации о БД

In [ ]:
queries = {
    '___DATABASES___': 'SELECT * FROM pg_database',
    '___TABLES___': "SELECT relname FROM pg_class WHERE relkind='r' AND relname !~ '^(pg_|sql_)';",
    '\n___COLUMNS___': "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'seats';"
}

with psycopg2.connect(
    dbname='demo',
    user=os.environ['POSTGRESQL_USER'],
    password=os.environ['POSTGRESQL_PASSWORD'],
    host=POSTGRESQL_HOST
) as conn:
    for name, query in queries.items():
        print('\n', name)
        with conn.cursor() as cur:
            cur.execute(query)
            for row in cur.fetchall():
                print(row)

In [ ]:
tables_db = []
query = "SELECT relname FROM pg_class WHERE relkind='r' AND relname !~ '^(pg_|sql_)';"
with psycopg2.connect(
    dbname='demo',
    user=os.environ['POSTGRESQL_USER'],
    password=os.environ['POSTGRESQL_PASSWORD'],
    host=POSTGRESQL_HOST
) as conn:
    with conn.cursor() as cur:
        cur.execute(query)
        for row in cur.fetchall():
            tables_db.append(row[0])

print(tables_db)

---
## Задание 1:
Для каждой таблицы БД выведите её структуру (название колонок).  
В текстовой ячейке представьте описание каждой таблицы — её назначение и структуру.

In [ ]:
with psycopg2.connect(
    dbname='demo',
    user=os.environ['POSTGRESQL_USER'],
    password=os.environ['POSTGRESQL_PASSWORD'],
    host=POSTGRESQL_HOST
) as conn:
    with conn.cursor() as cur:
        for table in tables_db:
            cur.execute("""
                SELECT column_name
                FROM information_schema.columns
                WHERE table_name = %s
                ORDER BY ordinal_position
            """, (table,))
            columns = [row[0] for row in cur.fetchall()]
            print(f"Таблица '{table}': {columns}")

### Описание таблиц БД «Авиаперевозки»

**boarding_passes** — посадочные талоны.  
Поля: `ticket_no` (номер билета), `flight_id` (ID рейса), `boarding_no` (номер посадки), `seat_no` (место).

**aircrafts_data** — справочник воздушных судов.  
Поля: `aircraft_code` (код ИАТА), `model` (модель, JSONB), `range` (макс. дальность полёта, км).

**flights** — рейсы.  
Поля: `flight_id`, `flight_no`, `scheduled_departure`, `scheduled_arrival`, `departure_airport`, `arrival_airport`, `status`, `aircraft_code`, `actual_departure`, `actual_arrival`.

**airports_data** — справочник аэропортов.  
Поля: `airport_code`, `airport_name` (JSONB), `city` (JSONB), `coordinates`, `timezone`.

**seats** — места в самолётах.  
Поля: `aircraft_code`, `seat_no`, `fare_conditions` (Business / Comfort / Economy).

**tickets** — билеты.  
Поля: `ticket_no`, `book_ref`, `passenger_id`, `passenger_name`, `contact_data`.

**ticket_flights** — сегменты перелёта (связь билета и рейса).  
Поля: `ticket_no`, `flight_id`, `fare_conditions`, `amount`.

**bookings** — бронирования.  
Поля: `book_ref`, `book_date`, `total_amount`.

## Пример использования магической команды %sql

In [ ]:
%load_ext sql

In [ ]:
CONNECT_DATA = 'postgresql://{}:{}@{}/{}'.format(
    os.environ['POSTGRESQL_USER'],
    os.environ['POSTGRESQL_PASSWORD'],
    POSTGRESQL_HOST,
    'demo'
)
print(f'Строка подключения готова: postgresql://reader:***@{POSTGRESQL_HOST}/demo')

In [ ]:
%%sql $CONNECT_DATA
SELECT column_name, udt_name
FROM information_schema.columns
WHERE table_name = 'boarding_passes'

---
## Задание 2:
Для каждой таблицы выведите информацию о типах её столбцов и количестве записей.  
Создайте словарь с названием таблиц и количеством записей.  
Найдите и выведите таблицу с максимальным количеством записей.

In [ ]:
table_counts = {}

with psycopg2.connect(
    dbname='demo',
    user=os.environ['POSTGRESQL_USER'],
    password=os.environ['POSTGRESQL_PASSWORD'],
    host=POSTGRESQL_HOST
) as conn:
    with conn.cursor() as cur:
        for table in tables_db:
            # Типы столбцов
            cur.execute("""
                SELECT column_name, udt_name
                FROM information_schema.columns
                WHERE table_name = %s
                ORDER BY ordinal_position
            """, (table,))
            cols = cur.fetchall()

            # Количество записей
            cur.execute(f'SELECT COUNT(*) FROM bookings.{table}')
            count = cur.fetchone()[0]
            table_counts[table] = count

            print(f"\n=== {table} (записей: {count}) ===")
            for col_name, col_type in cols:
                print(f"  {col_name}: {col_type}")

In [ ]:
print("Словарь таблица -> количество записей:")
print(table_counts)

max_table = max(table_counts, key=table_counts.get)
print(f"\nТаблица с максимальным количеством записей: '{max_table}' ({table_counts[max_table]} записей)")

---
## Задание 3:
Вывести названия тарифов, которые предлагают авиаперевозчики пассажирам.

In [ ]:
%%sql $CONNECT_DATA
SELECT DISTINCT fare_conditions AS "Тариф"
FROM bookings.ticket_flights
ORDER BY fare_conditions

**Комментарий:** Авиаперевозчик предлагает 3 тарифа: **Business** (бизнес-класс), **Comfort** (комфорт) и **Economy** (эконом).

---
## Задание 4:
По каждому тарифу найти общую сумму выручки за продажу билетов.

In [ ]:
%%sql $CONNECT_DATA
SELECT
    fare_conditions     AS "Тариф",
    SUM(amount)         AS "Общая выручка",
    COUNT(*)            AS "Количество билетов"
FROM bookings.ticket_flights
GROUP BY fare_conditions
ORDER BY SUM(amount) DESC

---
## Задание 5:
Какой тариф приносит максимальный доход?

In [ ]:
%%sql $CONNECT_DATA
SELECT
    fare_conditions AS "Тариф",
    SUM(amount)     AS "Максимальная выручка"
FROM bookings.ticket_flights
GROUP BY fare_conditions
ORDER BY SUM(amount) DESC
LIMIT 1

---
# Время выполнения запросов

Разные запросы требуют разное время на выполнение. Часто нужно оптимизировать запрос, либо находить и использовать другой инструмент для анализа данных.

In [ ]:
%time result = %sql SELECT * FROM bookings.ticket_flights LIMIT 10;

In [ ]:
result

---
## Задание 5 (часть 2):
Реализуйте 2 способами поиск модели самолёта с минимальной максимальной дальностью полёта.  
Выведите время выполнения каждого запроса и объясните, какой быстрее и почему.

In [ ]:
# Способ 1: подзапрос с MIN()
%time result1 = %sql SELECT model->>'ru' AS "Модель", range AS "Дальность (км)" FROM bookings.aircrafts_data WHERE range = (SELECT MIN(range) FROM bookings.aircrafts_data)

In [ ]:
result1

In [ ]:
# Способ 2: ORDER BY + LIMIT 1
%time result2 = %sql SELECT model->>'ru' AS "Модель", range AS "Дальность (км)" FROM bookings.aircrafts_data ORDER BY range ASC LIMIT 1

In [ ]:
result2

**Анализ:**

- **Способ 1 (MIN + подзапрос)** — PostgreSQL вычисляет минимум за один проход, затем находит запись. При наличии индекса на `range` — очень быстро.
- **Способ 2 (ORDER BY + LIMIT 1)** — сортирует всю таблицу, затем берёт первую строку. На больших таблицах без индекса — медленнее.

**Вывод:** Способ 1 предпочтительнее на больших таблицах, т.к. MIN() может использовать индекс без полной сортировки.

---
## Задание 6:
Выведите сколько рейсов имеют максимальную длительность полёта. Какова эта длительность?

In [ ]:
%%sql $CONNECT_DATA
SELECT
    MAX(scheduled_arrival - scheduled_departure)    AS "Максимальная длительность",
    COUNT(*) FILTER (
        WHERE (scheduled_arrival - scheduled_departure) =
              (SELECT MAX(scheduled_arrival - scheduled_departure) FROM bookings.flights)
    )                                               AS "Количество рейсов"
FROM bookings.flights

In [ ]:
%%sql $CONNECT_DATA
SELECT
    flight_no                                   AS "Номер рейса",
    departure_airport                           AS "Откуда",
    arrival_airport                             AS "Куда",
    scheduled_arrival - scheduled_departure     AS "Длительность"
FROM bookings.flights
WHERE (scheduled_arrival - scheduled_departure) = (
    SELECT MAX(scheduled_arrival - scheduled_departure) FROM bookings.flights
)
LIMIT 10

---
## Задание 7:
Выведите уникальные маршруты с максимальной длительностью полёта, включая:
- Код и название аэропорта отправления, город отправления
- Код и название аэропорта прибытия, город прибытия
- Длительность рейса

In [ ]:
%%sql $CONNECT_DATA
WITH max_duration AS (
    SELECT MAX(scheduled_arrival - scheduled_departure) AS max_dur
    FROM bookings.flights
),
routes AS (
    SELECT DISTINCT
        departure_airport,
        arrival_airport,
        MAX(scheduled_arrival - scheduled_departure) AS duration
    FROM bookings.flights
    GROUP BY departure_airport, arrival_airport
    HAVING MAX(scheduled_arrival - scheduled_departure) = (SELECT max_dur FROM max_duration)
)
SELECT
    r.duration                      AS "Длительность",
    r.departure_airport             AS "Код аэропорта отпр.",
    dep.airport_name->>'ru'         AS "Название аэропорта отпр.",
    dep.city->>'ru'                 AS "Город отправления",
    r.arrival_airport               AS "Код аэропорта приб.",
    arr.airport_name->>'ru'         AS "Название аэропорта приб.",
    arr.city->>'ru'                 AS "Город прибытия"
FROM routes r
JOIN bookings.airports_data dep ON dep.airport_code = r.departure_airport
JOIN bookings.airports_data arr ON arr.airport_code = r.arrival_airport
ORDER BY r.departure_airport

---
## Задание 8:
Определить аэропорт с максимальной нагрузкой по отправлениям и прибытиям. Вывести название и город.

In [ ]:
%%sql $CONNECT_DATA
WITH airport_load AS (
    SELECT departure_airport AS airport_code, COUNT(*) AS cnt
    FROM bookings.flights
    GROUP BY departure_airport

    UNION ALL

    SELECT arrival_airport AS airport_code, COUNT(*) AS cnt
    FROM bookings.flights
    GROUP BY arrival_airport
),
total_load AS (
    SELECT airport_code, SUM(cnt) AS total
    FROM airport_load
    GROUP BY airport_code
)
SELECT
    t.airport_code          AS "Код аэропорта",
    a.airport_name->>'ru'   AS "Название аэропорта",
    a.city->>'ru'           AS "Город",
    t.total                 AS "Всего рейсов"
FROM total_load t
JOIN bookings.airports_data a ON a.airport_code = t.airport_code
ORDER BY t.total DESC
LIMIT 1

---
## Задание 9:
Вывести среднее количество мест в самолётах по каждому классу обслуживания. Две цифры после запятой.

In [ ]:
%%sql $CONNECT_DATA
WITH seats_per_aircraft AS (
    SELECT
        aircraft_code,
        fare_conditions,
        COUNT(*) AS seat_count
    FROM bookings.seats
    GROUP BY aircraft_code, fare_conditions
)
SELECT
    fare_conditions                 AS "fare_conditions",
    ROUND(AVG(seat_count), 2)       AS "avg_seat_count"
FROM seats_per_aircraft
GROUP BY fare_conditions
ORDER BY fare_conditions

---
## Задание 10:
Найти самый дорогой рейс по суммарной выручке. Вывести: flight_id, final_amount, аэропорт и город отправления/прибытия.  
Использовать EXPLAIN ANALYZE. Сколько рейсов с максимальной суммой выручки?

In [ ]:
%%sql $CONNECT_DATA
WITH flight_revenue AS (
    SELECT
        flight_id,
        SUM(amount) AS final_amount
    FROM bookings.ticket_flights
    GROUP BY flight_id
),
max_revenue AS (
    SELECT MAX(final_amount) AS max_amount FROM flight_revenue
)
SELECT
    fr.flight_id                    AS "flight_id",
    fr.final_amount                 AS "final_amount",
    dep.airport_name->>'ru'         AS "departure_airport",
    dep.city->>'ru'                 AS "departure_city",
    arr.airport_name->>'ru'         AS "arrival_airport",
    arr.city->>'ru'                 AS "arrival_city"
FROM flight_revenue fr
JOIN bookings.flights f     ON f.flight_id = fr.flight_id
JOIN bookings.airports_data dep ON dep.airport_code = f.departure_airport
JOIN bookings.airports_data arr ON arr.airport_code = f.arrival_airport
WHERE fr.final_amount = (SELECT max_amount FROM max_revenue)
ORDER BY fr.flight_id

In [ ]:
%%sql $CONNECT_DATA
WITH flight_revenue AS (
    SELECT flight_id, SUM(amount) AS final_amount
    FROM bookings.ticket_flights
    GROUP BY flight_id
)
SELECT COUNT(*) AS "Кол-во рейсов с макс. выручкой"
FROM flight_revenue
WHERE final_amount = (SELECT MAX(final_amount) FROM flight_revenue)

In [ ]:
# EXPLAIN ANALYZE — план выполнения запроса
with psycopg2.connect(
    dbname='demo',
    user=os.environ['POSTGRESQL_USER'],
    password=os.environ['POSTGRESQL_PASSWORD'],
    host=POSTGRESQL_HOST
) as conn:
    with conn.cursor() as cur:
        cur.execute("""
            EXPLAIN ANALYZE
            WITH flight_revenue AS (
                SELECT flight_id, SUM(amount) AS final_amount
                FROM bookings.ticket_flights
                GROUP BY flight_id
            )
            SELECT
                fr.flight_id,
                fr.final_amount,
                dep.airport_name->>'ru' AS departure_airport,
                dep.city->>'ru'         AS departure_city,
                arr.airport_name->>'ru' AS arrival_airport,
                arr.city->>'ru'         AS arrival_city
            FROM flight_revenue fr
            JOIN bookings.flights f         ON f.flight_id = fr.flight_id
            JOIN bookings.airports_data dep ON dep.airport_code = f.departure_airport
            JOIN bookings.airports_data arr ON arr.airport_code = f.arrival_airport
            WHERE fr.final_amount = (SELECT MAX(final_amount) FROM flight_revenue)
        """)
        print("=== EXPLAIN ANALYZE ===")
        for row in cur.fetchall():
            print(row[0])

**Анализ EXPLAIN ANALYZE:**

- **Seq Scan** на `ticket_flights` — полное сканирование таблицы. Рекомендация: добавить индекс на `flight_id`.
- **HashAggregate** — группировка по `flight_id`. Эффективна при отсутствии индекса.
- **Hash Join** — соединение через хеш, хорошо работает для средних таблиц.
- **CTE вычисляется дважды** — для основного запроса и для подзапроса MAX. Оптимизация: вынести MAX в отдельное вычисление.

**Рекомендации по оптимизации:**
1. `CREATE INDEX ON bookings.ticket_flights(flight_id)` — убирает Seq Scan
2. Использовать оконную функцию `RANK()` вместо двойного подзапроса MAX
3. Индекс на `flights(flight_id)` уже есть (первичный ключ) — JOIN с flights эффективен

---
## Дополнительное задание:
Реализуйте 3 запроса для выявления узких мест работы авиаперевозчика.

In [ ]:
%%sql $CONNECT_DATA
-- Запрос 1: Аэропорты с высокой долей отменённых и задержанных рейсов
SELECT
    f.departure_airport                                             AS "Аэропорт",
    a.city->>'ru'                                                   AS "Город",
    COUNT(*)                                                        AS "Всего рейсов",
    COUNT(*) FILTER (WHERE f.status = 'Cancelled')                  AS "Отменено",
    COUNT(*) FILTER (WHERE f.status = 'Delayed')                    AS "Задержано",
    ROUND(
        100.0 * COUNT(*) FILTER (WHERE f.status IN ('Cancelled','Delayed')) / COUNT(*), 2
    )                                                               AS "% проблемных"
FROM bookings.flights f
JOIN bookings.airports_data a ON a.airport_code = f.departure_airport
GROUP BY f.departure_airport, a.city
HAVING COUNT(*) > 100
ORDER BY "% проблемных" DESC
LIMIT 10

In [ ]:
%%sql $CONNECT_DATA
-- Запрос 2: Рейсы с низкой заполняемостью (потенциально убыточные)
WITH seat_capacity AS (
    SELECT aircraft_code, COUNT(*) AS total_seats
    FROM bookings.seats
    GROUP BY aircraft_code
),
tickets_sold AS (
    SELECT flight_id, COUNT(*) AS sold
    FROM bookings.ticket_flights
    GROUP BY flight_id
)
SELECT
    f.departure_airport             AS "Откуда",
    f.arrival_airport               AS "Куда",
    sc.total_seats                  AS "Мест в самолёте",
    COALESCE(ts.sold, 0)            AS "Продано билетов",
    ROUND(
        100.0 * COALESCE(ts.sold, 0) / sc.total_seats, 2
    )                               AS "Заполняемость %"
FROM bookings.flights f
JOIN seat_capacity sc ON sc.aircraft_code = f.aircraft_code
LEFT JOIN tickets_sold ts ON ts.flight_id = f.flight_id
WHERE f.status IN ('Arrived', 'Departed')
ORDER BY "Заполняемость %" ASC
LIMIT 10

In [ ]:
%%sql $CONNECT_DATA
-- Запрос 3: Среднее опоздание вылета по моделям самолётов
SELECT
    ac.model->>'ru'                                         AS "Модель самолёта",
    COUNT(*)                                                AS "Рейсов с опозданием",
    ROUND(
        AVG(
            EXTRACT(EPOCH FROM (f.actual_departure - f.scheduled_departure)) / 60
        )::numeric, 1
    )                                                       AS "Среднее опоздание (мин)",
    MAX(
        EXTRACT(EPOCH FROM (f.actual_departure - f.scheduled_departure)) / 60
    )                                                       AS "Макс. опоздание (мин)"
FROM bookings.flights f
JOIN bookings.aircrafts_data ac ON ac.aircraft_code = f.aircraft_code
WHERE f.actual_departure IS NOT NULL
  AND f.actual_departure > f.scheduled_departure
GROUP BY ac.model
ORDER BY "Среднее опоздание (мин)" DESC

**Выводы по дополнительным запросам:**

**Запрос 1 — Проблемные аэропорты:** аэропорты с высокой долей отменённых/задержанных рейсов указывают на перегруженность инфраструктуры или системные проблемы в регионе.

**Запрос 2 — Низкая заполняемость:** маршруты с менее 50–60% заполняемостью убыточны. Рекомендация — сократить частоту рейсов или пересмотреть цены.

**Запрос 3 — Задержки по моделям:** систематические задержки конкретной модели самолёта — сигнал о проблемах с техническим обслуживанием или нехватке резервных бортов.